**Install Required Libraries**

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load your dataset
import pandas as pd
df = pd.read_csv("/content/sinhala sentences for grammar checker.csv")


# Split the data into training, validation, and test sets
train_data, temp_data = train_test_split(df, test_size=0.2, stratify=df["Label"], random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data["Label"], random_state=42)

# Save a dummy test_data.csv file
test_data.to_csv("test_data.csv", index=False)  # Create the missing file

# Verify the file was created correctly
test_data = pd.read_csv("test_data.csv")
print("Test data loaded successfully with columns:", test_data.columns)


# Initialize tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define custom dataset class (if not already defined)
class SinhalaGrammarDataset(torch.utils.data.Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        label = self.labels[index]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Create datasets and dataloaders
max_len = 128
batch_size = 16

train_dataset = SinhalaGrammarDataset(train_data["Sentence"].tolist(), train_data["Label"].tolist(), tokenizer, max_len)
val_dataset = SinhalaGrammarDataset(val_data["Sentence"].tolist(), val_data["Label"].tolist(), tokenizer, max_len)
test_dataset = SinhalaGrammarDataset(test_data["Sentence"].tolist(), test_data["Label"].tolist(), tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Optimizer
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
epochs = 5
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Validation
model.eval()
all_preds = []
all_labels = []

for batch in val_loader:
    with torch.no_grad():
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Classification report
print(classification_report(all_labels, all_preds))


Test data loaded successfully with columns: Index(['Sentence', 'Label'], dtype='object')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5, Loss: 0.5478991940617561
Epoch 2/5, Loss: 0.12245122909181468
Epoch 3/5, Loss: 0.02572027115282886
Epoch 4/5, Loss: 0.023974928501917493
Epoch 5/5, Loss: 0.031930647012220856
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        85
           1       1.00      1.00      1.00        98

    accuracy                           1.00       183
   macro avg       1.00      1.00      1.00       183
weighted avg       1.00      1.00      1.00       183



In [ ]:
# Test the model
model.eval()
test_preds = []
test_labels = []

for batch in test_loader:
    with torch.no_grad():
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1)

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Classification report for the test dataset
from sklearn.metrics import classification_report
print("Test Set Evaluation:")
print(classification_report(test_labels, test_preds))


Test Set Evaluation:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        85
           1       1.00      1.00      1.00        98

    accuracy                           1.00       183
   macro avg       1.00      1.00      1.00       183
weighted avg       1.00      1.00      1.00       183



In [70]:
def grammar_check(sentence, tokenizer, model, device):
    model.eval()
    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        pred = torch.argmax(outputs.logits, dim=-1).item()

    return pred

# Test with custom sentences
test_sentences = [
    "අපි කඩේ යයි",
    "අපි උදේට කඩෙන කමු.",
    "මම අද ගදර යයි.",
    "අපි හෙට ගමට යම.",
    "අපි හෙට නුවර යමි.",
    "මම දැන් වැවට යමි.",
    "මම ඔයාට අදරය කරයි.",
    "මම ඔයාගේ ගෙදර යයි.",
    "අපි උත්සාහයෙන් වැඩ කරමි.",
    "අපි කොහොමහරි දිනයි."
]

for sentence in test_sentences:
    result = grammar_check(sentence, tokenizer, model, device)
    if result == 1:
        print(f"'{sentence}' -> Correct grammar")
    else:
        print(f"'{sentence}' -> Incorrect grammar: Suggest correction")


'අපි කඩේ යයි' -> Incorrect grammar: Suggest correction
'අපි උදේට කඩෙන කමු.' -> Incorrect grammar: Suggest correction
'මම අද ගදර යයි.' -> Incorrect grammar: Suggest correction
'අපි හෙට ගමට යම.' -> Incorrect grammar: Suggest correction
'අපි හෙට නුවර යමි.' -> Incorrect grammar: Suggest correction
'මම දැන් වැවට යමි.' -> Incorrect grammar: Suggest correction
'මම ඔයාට අදරය කරයි.' -> Incorrect grammar: Suggest correction
'මම ඔයාගේ ගෙදර යයි.' -> Incorrect grammar: Suggest correction
'අපි උත්සාහයෙන් වැඩ කරමි.' -> Incorrect grammar: Suggest correction
'අපි කොහොමහරි දිනයි.' -> Incorrect grammar: Suggest correction


In [ ]:
# Save the fine-tuned model
model_save_path = "/content/sinhala_grammar_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model saved at {model_save_path}")


Model saved at /content/sinhala_grammar_model
